In [20]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from helpers import *
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [35]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)
x_train, x_test, y_train, y_test = split_data(x, y, 0.5)
x_train.shape

(125000, 30)

Reduce dimensions and standardize the columns of X

In [37]:
x_train_red = x_train[:, [0, 1, 2, 3, 8, 9, 10, 11, 13, 19, 21, 22, 29]]
x_test_red = x_test[:, [0, 1, 2, 3, 8, 9, 10, 11, 13, 19, 21, 22, 29]]

x_train_red_norm = standardize(x_train_red)
x_test_red_norm = standardize(x_test_red)

Running logistic regression with different values of degrees (polynomial feature expansion), gamma and different initial weights

In [39]:
number_of_w = 10

for degree in range(1, 5):
    tx_train = build_poly(x_train_red_norm, degree)
    tx_test = build_poly(x_test_red_norm, degree)
    for gamma in np.array([0.001, 0.01, 0.05, 0.1, 0.5, 0.8]):
        print("Running logistic regression with polynomial feature expansion with degree", degree, 
                 "and gamma", gamma,", with", number_of_w, "different initial weights.")
        for i in  range(number_of_w):
            w_init = np.random.rand(tx_train.shape[1], 1)
            #print(y_train.shape, tx_train.shape, w_init.shape)
            w, loss = logistic_regression(y_train, tx_train, w_init, 5000, gamma)
            print(i,":")
            print("Loss on training set", compute_accuracy(y_train, tx_train, w))
            print("Loss on test set:, ", compute_accuracy(y_test, tx_test, w))

## Generate predictions and save ouput in csv format for submission:


In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [346]:
a = np.random.rand(30)
np.around(a)

array([1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [347]:
a

array([0.98948792, 0.67934654, 0.34673991, 0.48885077, 0.5348651 ,
       0.08436905, 0.02071519, 0.18776387, 0.52188862, 0.85832073,
       0.50843456, 0.60499752, 0.84728409, 0.93295583, 0.00486189,
       0.66121463, 0.3157777 , 0.6534353 , 0.12888288, 0.54033186,
       0.44024959, 0.37952039, 0.0833081 , 0.55721317, 0.36309742,
       0.2762448 , 0.46041519, 0.00914704, 0.35695435, 0.45490402])